In [38]:
import os
import requests
from requests.auth import HTTPBasicAuth
import json
import datetime
from shapely.geometry import shape
from sentinelhub import SHConfig,SentinelHubBYOC, SentinelHubDownloadClient, SentinelHubRequest, DataCollection, MimeType,Geometry, CRS, BBox, bbox_to_dimensions
from sentinelhub_downloader import SentinelHubAPI, Config
import dateutil
from datetime import datetime, timedelta
import numpy as np

from planet import Planet
from planet import Session
from planet.clients import FeaturesClient
import planet.subscription_request
from planet.subscription_request import build_request

In [4]:

# --- User Inputs ---

# Collection ID of the Feature Collection with a single AOI, with quota reserved for
# the PV you're generating. 
PLANET_FEATURE_COLLECTION_ID = ""

# Your Planet API Key. You can get this from https://www.planet.com/account/#/user-settings 
# or if you have done `pip install planet` then you can type `planet auth value` and
# copy the results.
PLANET_API_KEY = ""

# Sentinel Hub Client ID and Secret. Go to https://apps.sentinel-hub.com/dashboard/#/account/settings
# and hit '+ Create' under OAuth Clients. Be sure to store these, you won't see 'secret' again.
SENTINEL_HUB_CLIENT_ID = ""
SENTINEL_HUB_CLIENT_SECRET = ""


# This name is used to identify the subscription, customize it to be a meaningful name
# so you can easily identify it later.
SUBSCRIPTION_BASE_NAME = ""


# Customize these values for the PV you want to download.
PV_VAR_TYPE="biomass_proxy"
PV_VAR_ID="BIOMASS-PROXY_V4.0_10"
# Other common options for SWC are: SWC-AMSR2-C_V2.0_100, SWC-AMSR2-X_V2.0_100, SWC-SMAP-L_V5.0_1000
# Full list of options: https://developers.planet.com/docs/planetary-variables/soil-water-content-technical-specification/

# These are the biomass proxy options.
#PV_VAR_TYPE="biomass_proxy"  # PV type for Biomass Proxy
PV_VAR_ID="BIOMASS-PROXY_V4.0_10"

# Bands
BANDS="CB"

# Start date for the data you want to download.
SUB_START_TIME = "2024-04-01"

# Comment out either an end date or NO_END_TIME to make an ongoing subscription
SUB_END_TIME = "2024-04-07"
#SUB_END_TIME = "NO_END_TIME"

# Define output folder for TIFF files
# Change this to the path of the folder you want to save the TIFF files to
OUTPUT_FOLDER = "Local/Path/To/Download/Folder"


# Output template for the tiff files that result. The template supports the following variables
# {byoc_id} - The BYOC collection ID
# {date} - The date of the image
# {id} - A unique identifier for the image
OUTPUT_TEMPLATE = "Filename_{date}.tiff" 

# different template examples
# OUTPUT_TEMPLATE = "SWC_{date}"  # Just date
# OUTPUT_TEMPLATE = "SWC_Alberta_Region1_{date}_{id}"  # With ID
# OUTPUT_TEMPLATE = "{byoc_id}_SWC_Alberta_{date}_{id}.tiff"


In [3]:
##defining base url
subscriptions_url = "https://api.planet.com/subscriptions/v1"

plsdk_auth = "YOUR_PLANET_API_KEY"
auth = planet.Auth.from_key(key=PLANET_API_KEY)
sess = planet.Session(auth)
pl = planet.Planet(sess)

In [5]:
#Function to create bulk subscriptions for multi-polygon collections

def create_planet_subscription_bulk():
    start_time = datetime.strptime(SUB_START_TIME, "%Y-%m-%d")
    end_time = datetime.strptime(SUB_END_TIME, "%Y-%m-%d")
    
    subscription_name = SUBSCRIPTION_BASE_NAME
    
    pv_source = planet.subscription_request.subscription_source(
    source_id=PV_VAR_ID,
    geometry={
        "content": "pl:features/my/" + PLANET_FEATURE_COLLECTION_ID,
        "type": "ref",
    },
    start_time=start_time,
    end_time=end_time,
)
    
    payload = planet.subscription_request.build_request(
        name=subscription_name,
        source=pv_source,
        hosting = planet.subscription_request.sentinel_hub(
             collection_id="",
             create_configuration=True
        ))
    
    subs = [payload]
    
    try:
        result = pl.subscriptions.bulk_create_subscriptions(subs)
        print(result)
        return result
    except Exception as e:
        print(f"❌ Error creating subscription '{subscription_name}': {str(e)}")
        raise

# Create subscriptions for each item
subreq = create_planet_subscription_bulk()


{'_links': {'list': 'https://api.planet.com/subscriptions/v1?created=2025-11-26T20%3A28%3A55Z%2F..&geom_ref=pl%3Afeatures%2Fmy%2Fingested-from-planet_mb_sites_smallshp-Arn2eQr&name=MB_MultiTest'}}


In [ ]:
#Now you need to wait for the subscription to process. The next steps will fail if the sentinelhub collection isn't available.

In [27]:
#Get info from the subscription request
headers = {"Authorization": f"api-key {PLANET_API_KEY}"}
resp = requests.get(subreq['_links']['list'], headers=headers)
subs = resp.json()


#All items will share the same collection ID, so we can just read into the first one to get it
COLLECTION_ID = subs['subscriptions'][0]['hosting']['parameters']['collection_id']

In [41]:
#Get the tiles from the collection
config = SHConfig(sh_client_id=SENTINEL_HUB_CLIENT_ID, sh_client_secret=SENTINEL_HUB_CLIENT_SECRET)
byoc = SentinelHubBYOC(config)

tiles = list(byoc.iter_tiles(COLLECTION_ID))

60

In [42]:
#You'll most likely want to iterate across your tiles
tile = tiles[0]

In [44]:
#Download the tile

tile_geom = Geometry(tile['coverGeometry'], crs=CRS(tile['coverGeometry']['crs']['properties']['name']))
tile_time = tile['sensingTime']

data_collection = DataCollection.define_byoc(COLLECTION_ID)

resolution = 10

evalscript = """
//VERSION=3
function setup() {
  return {
    input: [{bands: ["CB"]}],
    output: { bands: 1 },
  };
}

function evaluatePixel(sample) {
  return [(1/256)*sample.CB];
}
"""

request = SentinelHubRequest(
    evalscript=evalscript,
    input_data=[
        SentinelHubRequest.input_data(
            data_collection=data_collection,
            time_interval=(tile_time, tile_time)
        )
    ],
    responses=[
        SentinelHubRequest.output_response("default", MimeType.TIFF)
    ],
    geometry=tile_geom,
    size = bbox_to_dimensions(tile_geom.bbox, resolution=resolution),
    data_folder = OUTPUT_FOLDER,
    config=config
)

result = request.get_data(save_data=True)